# Pipeline example: mortality, fertility and immigration

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import time

from vivarium import InteractiveContext
from vivarium.framework.configuration import build_simulation_configuration
from vivarium.config_tree import ConfigTree

from vivarium_public_health.population import FertilityAgeSpecificRates
from vivarium_public_health.population import Mortality
from vivarium_public_health.population import Emigration
from vivarium_public_health.population import ImmigrationDeterministic as Immigration

from vivarium_public_health.population.spenser_population import TestPopulation
from vivarium_public_health.population.spenser_population import build_mortality_table
from vivarium_public_health.population.spenser_population import transform_rate_table
from vivarium_public_health.population.spenser_population import prepare_dataset
from vivarium_public_health.population.spenser_population import compute_migration_rates

from vivarium_public_health.utilities import read_config_file

## Base plugins for simulation

In [ ]:
def base_plugins_simulation():
    config = {'required': {
                  'data': {
                      'controller': 'vivarium_public_health.testing.mock_artifact.MockArtifactManager',
                      'builder_interface': 'vivarium.framework.artifact.ArtifactInterface'
                  }
             }
    }

    return ConfigTree(config)

## Configure a simulation

In [ ]:
def config_simulation(inp_file):
    
    if inp_file['prepare_data']['prepare']:
        # read a dataset (normally from daedalus), change columns to be readable by vivarium
        # this function creates a file saved at output_path
        prepare_dataset(
            dataset_path=inp_file['prepare_data']['path_to_dataset'], 
            output_path=inp_file['prepare_data']['path_to_output'],
            lookup_ethnicity=inp_file['prepare_data']['path_to_lookup_ethnicity'],
            columns_map=inp_file['prepare_data']['columns_map'],
            location_code=inp_file['prepare_data']['location_code']
        )
    
    # ============= CONFIGURATION
    if inp_file['configuration']['population']['population_size'] <= 0:
        pop_size = len(pd.read_csv(inp_file['paths']['path_to_pop_file']))
    else:
        pop_size = inp_file['configuration']['population']['population_size']
    
    # config object
    config = build_simulation_configuration()
    config.update({
        'time': {
            'start': inp_file['configuration']['time']['start'],
            'end': inp_file['configuration']['time']['end'],
            'step_size': inp_file['configuration']['time']['step_size']
        },
        'randomness': inp_file['configuration']['randomness'],
        'input_data': inp_file['configuration']['input_data'],
    }, 
        layer='model_override')
    
    config.update({
        'path_to_pop_file': inp_file['paths']['path_to_pop_file'],
        'path_to_mortality_file': inp_file['paths']['path_to_mortality_file'],
        'path_to_fertility_file': inp_file['paths']['path_to_fertility_file'],
        'path_to_emigration_file': inp_file['paths']['path_to_emigration_file'],
        'path_to_immigration_file': inp_file['paths']['path_to_immigration_file'],
        'path_to_total_population_file': inp_file['paths']['path_to_total_population_file'],
        
        'population': {
            'population_size': pop_size,
            'age_start': inp_file['configuration']['population']['age_start'],
            'age_end': inp_file['configuration']['population']['age_end'],
        },
        },
    )
    return config

## Create an interactive context manager

In [ ]:
inp_file = read_config_file("../config/model_specification_pipeline_005.yaml")

base_plugins = base_plugins_simulation()
config = config_simulation(inp_file=inp_file)
components = [eval(x) for x in inp_file["list_components"]]
simulation = InteractiveContext(components=components,
                                configuration=config,
                                plugin_configuration=base_plugins,
                                setup=False)

## Mortality rates

In [ ]:
components

In [ ]:
df = pd.read_csv(config.path_to_mortality_file)
# to save time, only look at locatiosn existing on the test dataset.
mortality_rate_df = df[(df['LAD.code']=='E09000002')]

asfr_data = transform_rate_table(mortality_rate_df,
                                      2011,
                                      2012,
                                      config.population.age_start,
                                      config.population.age_end)

simulation._data.write("cause.all_causes.cause_specific_mortality_rate", asfr_data)

In [ ]:
mortality_rate_df

## Fertility rates


In [ ]:
df = pd.read_csv(config.path_to_fertility_file)
# to save time, only look at locatiosn existing on the test dataset.
fertility_rate_df = df[(df['LAD.code']=='E09000002')]

asfr_data_fertility = transform_rate_table(fertility_rate_df,
                                      2011,
                                      2012,
                                      10,  # starting age for fertility
                                      50,  # finishing age for fertility
                                      [2]) # gender (only females [2])


simulation._data.write("covariate.age_specific_fertility_rate.estimate", asfr_data_fertility)


In [ ]:
asfr_data_fertility.head()

In [ ]:
fertility_rate_df

In [ ]:
simulation.name

## Emigration

In [ ]:
# setup emigration rates
df_emigration = pd.read_csv(config.path_to_emigration_file)
df_total_population = pd.read_csv(config.path_to_total_population_file)

df_emigration = df_emigration[
    (df_emigration['LAD.code'] == 'E09000002')]
df_total_population = df_total_population[
    (df_total_population['LAD'] == 'E09000002')]

asfr_data_emigration = compute_migration_rates(df_emigration, df_total_population, 
                                               2011, 
                                               2012, 
                                               config.population.age_start, 
                                               config.population.age_end)

In [ ]:
asfr_data_emigration.head()

In [ ]:
# Population (total)
plt.figure(figsize=(15, 10))
plt.plot(asfr_data_emigration["age_start"], 
         asfr_data_emigration["mean_value"], 
         '.', c='k', lw=4, markersize=12)
plt.xlabel("Age", size=32)
plt.ylabel("Emigration rate", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
plt.grid()
plt.show()

In [ ]:
simulation._data.write("covariate.age_specific_migration_rate.estimate", asfr_data_emigration)

## Immigration rates

In [ ]:
# setup immigration rates
df_immigration = pd.read_csv(config.path_to_immigration_file)

df_immigration = df_immigration[
    (df_immigration['LAD.code'] == 'E09000002')]

asfr_data_immigration = compute_migration_rates(df_immigration, df_total_population, 
                                                2011, 
                                                2012, 
                                                config.population.age_start, 
                                                config.population.age_end,
                                                normalize=False
                                               )

In [ ]:
# TESTING
# asfr_data_immigration.loc[asfr_data_immigration["age_start"] > 80, "mean_value"] = 100
asfr_data_immigration["mean_value"].describe()

In [ ]:
# XXX CHECK!!!
total_immigrants = df_immigration[df_immigration.columns[4:]].sum().sum()

In [ ]:
# Population (total)
plt.figure(figsize=(15, 10))
plt.plot(asfr_data_immigration["age_start"], 
         asfr_data_immigration["mean_value"], 
         '.', c='k', lw=4, markersize=12)
plt.xlabel("Age", size=32)
plt.ylabel("Immigration rate", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
plt.grid()
plt.show()

In [ ]:
simulation._data.write("cause.all_causes.cause_specific_immigration_rate", asfr_data_immigration)
simulation._data.write("cause.all_causes.cause_specific_total_immigrants_per_year", total_immigrants)

## Setup a simulation and run for `num_days`

In [ ]:
sim_start = time.time()

simulation.setup()
num_days = 365*5 + 10
simulation.run_for(duration=pd.Timedelta(days=num_days))



sim_end = time.time()

In [ ]:
pop = simulation.get_population()

print(f"Total time: {sim_end - sim_start}")
print (f'#alive: {len(pop[pop["alive"]=="alive"])}')
print (f'#dead: {len(pop[pop["alive"]!="alive"])}')
try:
    print (f'#new borns: {len(pop[pop["parent_id"]!=-1])}')
except Exception:
    print("Fertility component should be activated to print new borns!")

In [ ]:
pop["cause_of_death"].value_counts()

In [ ]:
pop[pop["cause_of_death"] == "all_causes"].head()

## Plot results

In [ ]:
#min_time = pop["entrance_time"].min().strftime("%Y-%m-%d")
min_time = "2011-01-01"
max_time = datetime.datetime.strptime("2012-01-01", "%Y-%m-%d")

print("min_time:", min_time)
print("max_time:", max_time)

In [ ]:
# --- input
# intervals for plotting (in days)
interval_in_days = 10
# list of ethnicities
sel_ethnicity = ["WBI", "WHO"]

time_axis = []
population_axis = []
population_ETH_axis = []
population_M_axis = []
population_F_axis = []

curr_time = datetime.datetime.strptime(min_time, "%Y-%m-%d")
while curr_time <= max_time:
    time_axis.append(curr_time)
    
    # dead population until current time (changes in the while loop)
    pop_dead = pop[pop["exit_time"] <= curr_time.strftime("%Y-%m-%d")]
    curr_pop = pop[pop["entrance_time"] <= curr_time.strftime("%Y-%m-%d")]
    
    current_alive_population = len(curr_pop) - len(pop_dead)
    population_axis.append(current_alive_population)
    
    curr_pop_eth = len(curr_pop[curr_pop["ethnicity"].isin(sel_ethnicity)])
    current_alive_population_eth = curr_pop_eth - len(pop_dead[pop_dead["ethnicity"].isin(sel_ethnicity)])
    population_ETH_axis.append(current_alive_population_eth)
    
    curr_pop_male = len(curr_pop[curr_pop["sex"] == 1])
    current_alive_population_male = curr_pop_male - len(pop_dead[pop_dead["sex"] == 1])
    population_M_axis.append(current_alive_population_male)
    
    curr_pop_female = len(curr_pop[curr_pop["sex"] == 1])
    current_alive_population_female = curr_pop_female - len(pop_dead[pop_dead["sex"] == 2])
    population_F_axis.append(current_alive_population_female)
    
    # go to next time, according to the selected interval_in_days
    curr_time = datetime.datetime.strptime(curr_time.strftime("%Y-%m-%d"), "%Y-%m-%d")
    curr_time += datetime.timedelta(days=interval_in_days)

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# Population (total)
plt.figure(figsize=(15, 10))
plt.plot(time_axis, population_axis, 
         c='k', lw=4, marker="o")
plt.xlabel("Time", size=32)
plt.ylabel("Population", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
plt.grid()
plt.show()

In [ ]:
# Population (gender)
plt.figure(figsize=(15, 10))
plt.plot(time_axis, population_M_axis, 
         c='k', lw=4, marker="o", 
         label="Male"
        )
plt.plot(time_axis, population_F_axis, 
         c='r', lw=4, marker="o",
         label="Female"
        )
plt.xlabel("Time", size=32)
plt.ylabel("Population", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
plt.grid()
plt.legend(fontsize=24)
plt.show()

In [ ]:
# Population (ethnicity)
plt.figure(figsize=(15, 10))
plt.plot(time_axis, population_ETH_axis, 
         c='k', lw=4, marker="o", 
         label=sel_ethnicity
        )
plt.xlabel("Time", size=32)
plt.ylabel("Population", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
plt.grid()
plt.legend(fontsize=24)
plt.show()

## Histograms

In [ ]:
# only immigrants
pop_immig = pop[pop["entrance_time"] > curr_time.strftime("2011-01-01")]
#pop_immig = pop_immig[pop_immig["parent_id"] == -1]

In [ ]:
pop["ethnicity"].value_counts()

In [ ]:
pop_immig['ethnicity'].value_counts()

In [ ]:
series2plot = pop_immig['ethnicity'].value_counts() / pop["ethnicity"].value_counts() * 100.
indx = range(len(series2plot))

plt.figure(figsize=(15, 10))
plt.bar(indx, series2plot, color='k')

plt.xticks(indx, series2plot.index, size=24)
plt.yticks(size=24)
plt.xlabel("Ethnicity", size=32)
plt.ylabel("% of group population", size=32)

plt.grid()
#plt.legend(fontsize=24)
plt.show()

In [ ]:
series2plot = pop_immig['ethnicity'].value_counts() / len(pop_immig["ethnicity"]) * 100.
indx = range(len(series2plot))

plt.figure(figsize=(15, 10))
plt.bar(indx, series2plot, color='k')

plt.xticks(indx, series2plot.index, size=24)
plt.yticks(size=24)
plt.xlabel("Ethnicity", size=32)
plt.ylabel("% of total population", size=32)

plt.grid()
#plt.legend(fontsize=24)
plt.show()

In [ ]:
series2plot = pop_immig['sex'].value_counts() / len(pop_immig["sex"]) * 100.
indx = range(len(series2plot))

plt.figure(figsize=(15, 10))
plt.bar(indx, series2plot, color='k')

plt.xticks(indx, series2plot.index, size=24)
plt.yticks(size=24)
plt.xlabel("Gender", size=32)
plt.ylabel("%", size=32)

plt.grid()
#plt.legend(fontsize=24)
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

pop_immig["entrance_time"].hist(bins=int((365+10)/10), 
                           rwidth=0.9, 
                           color='k',
                           align='left'
                          )
plt.xlabel("Time", size=32)
plt.ylabel("Immigration", size=32)
plt.xticks(size=24, rotation=90)
plt.yticks(size=24)
#plt.legend(fontsize=24)
plt.show()

In [ ]:
pop